In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.masked import masked_tensor, as_masked_tensor
import pickle
from random import randint,sample
import time
import re
from copy import deepcopy

In [ ]:
A=torch.randn((4,4))
#print(A)
mask = torch.zeros_like(A).bool()
B = masked_tensor(A,mask)

#print(B.argmax(dim=1).sum())
print(mask.sum(dim=1).shape)
print((mask.sum(dim=1)==0).shape)

In [2]:
with open('nconstName.pickle', 'rb') as handle:
    nconstName = pickle.load(handle)

with open('nconst_idx.pickle', 'rb') as handle:
    nconst_idx = pickle.load(handle)

with open('nconst_universe.pickle', 'rb') as handle:
    nconst_universe = pickle.load(handle)

with open('tconst_idx.pickle', 'rb') as handle:
    tconst_idx = pickle.load(handle)

with open('tconst_universe.pickle', 'rb') as handle:
    tconst_universe = pickle.load(handle)

with open('idx_casts.pickle', 'rb') as handle:
    idx_casts = pickle.load(handle)

movie_idxs = list(idx_casts.keys())

with open('filmDict.pickle', 'rb') as handle:
    filmDict = pickle.load(handle)

onehot_casts = torch.load('onehot_casts.pt')

with open('rep_subset.pickle', 'rb') as handle:
    repSubset = pickle.load(handle)
repSubset = list(map(tconst_idx.get,repSubset))

In [ ]:
print(len(tconst_universe))
print(len(nconst_universe))

In [3]:
with open('formulabest300.txt',mode='r') as f:
    t = f.read()
formulabest300=re.findall(r'\((.*?)\)', t)
with open('c2n_500_attempt1.txt',mode='r') as f:
    t = f.read()
formulabest500=re.findall(r'\((.*?)\)', t)

In [ ]:
def get_cast(movie_index:int,onehot_casts:torch.Tensor=onehot_casts)->torch.Tensor:
    return  onehot_casts[movie_index]
mcast = get_cast(13111)#.flatten().tolist()

# for xsa in mcast:
#     print(f"{nconstName[nconst_idx[xsa]]}")
f300_as_indices = torch.tensor([tconst_idx[e] for e in formulabest300])
f300_onehot_casts = get_cast(f300_as_indices)
print(f300_onehot_casts.shape)
mcast_reshaped = mcast.unsqueeze(1)
print(mcast_reshaped.shape)
#f300_mcast_dotprods = torch.bmm(f300_onehot_casts,mcast_reshaped.transpose(1,2)).squeeze(-1)
f300_dps = torch.matmul(f300_onehot_casts,mcast_reshaped).squeeze(-1)
print(torch.nonzero(f300_dps).flatten())
tconstants = list(map(tconst_idx.get,f300_as_indices[torch.nonzero(f300_dps).flatten()].tolist()))
print(tconstants)
titles = [filmDict[tconst]["title"] for tconst in tconstants]
for t in titles:
    print(t)


In [ ]:
class cine2Nerdle:
    def __init__(self,nconstName,nconst_idx,nconst_universe,tconst_idx
                 ,tconst_universe,idx_casts,movie_idxs,onehot_casts=onehot_casts):
        self.nconstName = nconstName
        self.nconst_idx = nconst_idx
        self.nconst_universe = nconst_universe

        self.tconst_idx = tconst_idx
        self.tconst_universe = tconst_universe

        self.idx_casts = idx_casts
        self.movie_idxs = movie_idxs
        self.onehot_casts = onehot_casts
        
    def sample_movie_indexes(self,k=300):
        return
    
    def generate_subset_idxs(self,subset_dim=300,tconst_subset=[],batch_size=1):
        #returns a list of subset_dim movie indexes
        if tconst_subset == []:
            return torch.tensor([sample(self.movie_idxs,k=subset_dim) for _ in range(batch_size)],dtype=torch.long)
        else:
            return torch.tensor([[self.tconst_idx[tconst] for tconst in tconst_subset] for _ in range(batch_size)],dtype=torch.long)
    
    def subset_idxs_onehot(self,subset_indices):
        result = F.one_hot(subset_indices,num_classes=len(self.tconst_universe))
        return result.sum(dim=1,dtype=torch.float)

    def subset_idxs_onehot2(self,subset_idxs):
        #returns the onehot vector corresponding to the given subset
        #value 1 at each of the movie indices in the subset, 0 elsewhere
        subset_onehot = torch.zeros(size=(len(subset_idxs),len(self.tconst_universe)),dtype=torch.float)
        for i in range(len(subset_idxs)):
            for idx in subset_idxs:
                subset_onehot[i][idx] = 1
        return subset_onehot
    
    def movie_cast_onehot(self,movie_idx):
        #returns the onehot vector corresponding to a given movie index
        movie_onehot = torch.zeros(size=(len(movie_idx),len(self.nconst_universe)),dtype=torch.float)
        for i in range(len(movie_idx)):
            for idx in self.idx_casts[movie_idx]:
                movie_onehot[i][idx] = 1
        return movie_onehot
    
    def init_actorsPlayed(self,batch_size,batch=[]):
        #initializes the actorsPlayed vector to the onehot representation of the
        #starting movies' cast
        result = torch.zeros(size=(batch_size,len(self.nconst_universe)),dtype=torch.float)
        return result
    
    def init_previousMoviePlayed(self,batch)->torch.Tensor:
        #batch is a list of movie indices
        #within idx_casts, at each of these indices there is a list of actor indices
        #result = torch.tensor([F.one_hot(torch.tensor(self.idx_casts[idx]),num_classes=len(self.nconst_universe)) for idx in batch])
        return self.onehot_casts[batch].to(torch.float)#result

    def update_actorsPlayed(self,actorsPlayed,next_movie_cast_onehot):
        #updates the actorsPlayed vector by adding the next movies' cast onehot vector
        #to the actorsPlayed vector
        #might be faster to do this in-place? if possible
        return torch.add(actorsPlayed,next_movie_cast_onehot)
    
    def init_moviesPlayed(self,starting_movie_indices):
        #initializes the moviesPlayed onehot vector
        #1 at the starting movie index, 0 otherwise

        #starting_movie_indices is a 1D list of movie indexes
        result = torch.tensor([[x] for x in starting_movie_indices],dtype=torch.long)
        #print(result.shape)
        return self.subset_idxs_onehot(result)
    
    def update_moviesPlayed(self,moviesPlayed,next_movie_idx):
        #updates the moviesPlayed onehot vector by adding the onehot vector for the
        #next movie
        return torch.add(moviesPlayed,self.init_moviesPlayed(next_movie_idx))
    


In [ ]:
class Cine2NerdleAgent(nn.Module):
    def __init__(self, n_actors, n_movies, embedding_dim, subset_dim):
        super(Cine2NerdleAgent, self).__init__()
        #self.actorsPlayed_embedding = nn.Embedding(n_actors, embedding_dim)
        self.actorsPlayed_fc1 = nn.Linear(n_actors,embedding_dim)
        self.actorsPlayed_fc2 = nn.Linear(embedding_dim,embedding_dim)
        #self.previousMoviePlayed_embedding = nn.Embedding(n_actors, embedding_dim)
        self.previousMoviePlayed_fc1 = nn.Linear(n_actors, embedding_dim)
        self.previousMoviePlayed_fc2 = nn.Linear(embedding_dim,embedding_dim)
        #self.moviesPlayed_embedding = nn.Embedding(n_movies,embedding_dim)
        self.moviesPlayed_fc1 = nn.Linear(n_movies,embedding_dim)
        self.moviesPlayed_fc2 = nn.Linear(embedding_dim,embedding_dim)
        #self.playerSubset_embedding = nn.Embedding(n_movies,embedding_dim)
        self.playerSubset_fc1 = nn.Linear(n_movies,embedding_dim)
        self.playerSubset_fc2 = nn.Linear(embedding_dim,embedding_dim)
        self.fc1 = nn.Linear(embedding_dim * 4, subset_dim+1)
        self.fc2 = nn.Linear(subset_dim+1, subset_dim+1)
    
    def forward(self, subset, moviesPlayed, actorsPlayed, previousMoviePlayed,verbose=False)->torch.Tensor:
        subset_emb = torch.relu(self.playerSubset_fc1(subset))
        subset_emb = torch.relu(self.playerSubset_fc2(subset_emb))
            
        moviesPlayed_emb = torch.relu(self.moviesPlayed_fc1(moviesPlayed))
        moviesPlayed_emb = torch.relu(self.moviesPlayed_fc2(moviesPlayed_emb))
            
        actorsPlayed_emb = torch.relu(self.actorsPlayed_fc1(actorsPlayed))
        actorsPlayed_emb = torch.relu(self.actorsPlayed_fc2(actorsPlayed_emb))

            
        previousMoviePlayed_emb = torch.relu(self.previousMoviePlayed_fc1(previousMoviePlayed))
        previousMoviePlayed_emb = torch.relu(self.previousMoviePlayed_fc2(previousMoviePlayed_emb))

            
        x = torch.cat((subset_emb,moviesPlayed_emb,actorsPlayed_emb,previousMoviePlayed_emb), dim=-1)
            
        x = torch.relu(self.fc1(x))
            
        x = self.fc2(x)
        if verbose:
            print(f"subset_emb.shape={subset_emb.shape}")
            print(f"moviesPlayed_emb.shape={moviesPlayed_emb.shape}")
            print(f"actorsPlayed_emb.shape={actorsPlayed_emb.shape}")
            print(f"previousMoviePlayed_emb.shape={previousMoviePlayed_emb.shape}")
            print(f"after cat = {x.shape}")
            print(f"after fc1 = {x.shape}")
            print(f"after fc2 = {x.shape}")
        
        return x

In [ ]:
def move_cast_onehots(move, subset_indices, onehot_casts):
    batch_size = move.size(0)
    n_actors = onehot_casts.size(1)
    
    # Ensure all tensors are on the same device
    #move = move.to(onehot_casts.device)
    #subset_indices = subset_indices.to(onehot_casts.device)
    
    # Create a mask for values in move that are less than len(subset_indices)
    valid_mask = move < subset_indices.size(1)
    
    # Initialize the result tensor with zeros
    result = torch.zeros(batch_size, n_actors, dtype=onehot_casts.dtype)
    
    # Get the valid indices from subset_indices
    valid_indices = subset_indices[torch.arange(batch_size), move.clamp(max=subset_indices.size(1) - 1)]
    
    # Map the valid indices to onehot_casts
    result[valid_mask] = onehot_casts[valid_indices[valid_mask]]
    
    # Debugging prints
    # print("Move tensor:", move)
    # print("Subset indices tensor:", subset_indices)
    # print("Valid mask:", valid_mask)
    # print("Valid indices:", valid_indices)
    # print("Result tensor:", result)
    
    return result
def move_movie_onehots(move, subset_indices,onehot_casts):
    batch_size = move.size(0)
    onehot_vectors = torch.zeros(batch_size, onehot_casts.size(0))
    
    for i in range(batch_size):
        if move[i] == subset_indices.size(1):
            onehot_vectors[i] = torch.zeros(onehot_casts.size(0))
        else:
            #print(f"{i}, {subset_indices[i][move[i]]}")
            onehot_vectors[i, subset_indices[i][move[i]]] = 1
    
    return onehot_vectors

def batch_dot_product(tensor1, tensor2)->torch.Tensor:
    # Element-wise multiplication
    elementwise_product = tensor1 * tensor2
    # Summing along the last dimension to get the dot product for each batch
    dot_products = elementwise_product.sum(dim=1)
    return dot_products

def good_or_bad_concession(move:torch.Tensor, subset_indices, subset_onehot, moviesPlayed, actorsPlayed, previousMoviePlayed):
    result=torch.zeros_like(move,dtype=move.dtype).bool()
    for i in range(result.size(0)):
        if move[i] == subset_indices.size(1):
            result[i] = True
    
    #to determine if the agent concede correctly or incorrectly
    #we need to check if it still had a valid move
    #to do this we do something like movies_left = max(0,subset_onehot-moviesPlayed)
        #if the sum of this(dim=1) is 0, then the concede was correct for sure
    #if it is not 0
    # , then we need to find non-zero dot products between the agent's movies_left cast onehots and previousmoviePlayed
    #but unfortunately we will have to do this for each movie_left for each member of the batch unless we do some masking
    
    #so, for each member of the batch:
    #1. get the non-zero indices of movies_left
    #2. get the cast_onehot vectors from onehot_casts at the indices from 1
    #3. obtain the dotproduct of each of these cast vectors with the previousMoviePlayed element corresponding
    #   to the member of the batch. 
    #4. obtain the sum of each of these cast vectors with the actorsPlayed element corresponding to the 
    #   member of the batch
    #5. if for a given movie in movies_left both:
        #a. the dot product from 3. is non-zero
        #b. the sum from 4. does not contain any values greater than 3
        #then, the member of the batch conceded incorrectly and we will punish



    return result

In [ ]:
def subset_valid_mask(subset_cast_onehots:torch.Tensor,subset_movie_onehots:torch.Tensor
                      ,previousMoviePlayed:torch.Tensor,moviesPlayed:torch.Tensor
                      ,actorsPlayed:torch.Tensor)->torch.Tensor:
    #subset_cast_onehots shape is [batch_size, subset_size, n_actors]
    pMP_reshaped = previousMoviePlayed.unsqueeze(1)
    casts_pMP_dotprods = torch.bmm(subset_cast_onehots,pMP_reshaped.transpose(1,2)).squeeze(-1)
    #print(casts_pMP_dotprods.shape)
    casts_intersect_mask = casts_pMP_dotprods.bool()
    #print(casts_intersect_mask.sum(dim=1))
    mP_reshaped = moviesPlayed.unsqueeze(1)
    not_yet_played_mask = torch.logical_not(torch.bmm(subset_movie_onehots,mP_reshaped.transpose(1,2)).squeeze(-1).bool())
    aP_reshaped = actorsPlayed.unsqueeze(1)
    doesnt_violate_xs_mask = ((subset_cast_onehots+aP_reshaped) <= 3).all(dim=-1).bool()
    result = torch.logical_and(casts_intersect_mask,not_yet_played_mask)
    result = torch.logical_and(result,doesnt_violate_xs_mask)
    #output should be a boolean tensor of shape [batch_size,subset_size]
    #then we can mask the output of agent with this and the argmax will
    #be the valid action which the agent has selected
    return result, casts_pMP_dotprods

###need to make subset_cast_onehots and subset_movie_onehots

In [ ]:
test_game = cine2Nerdle(nconstName=nconstName,nconst_idx=nconst_idx,nconst_universe=nconst_universe
                        ,tconst_idx=tconst_idx,tconst_universe=tconst_universe,idx_casts=idx_casts
                        ,movie_idxs=movie_idxs)

agent = Cine2NerdleAgent(n_actors=len(nconst_universe),n_movies=len(tconst_universe),
                         embedding_dim=500,subset_dim=300)
optimizer = optim.SGD(agent.parameters(),lr=1e-3)
loss_fn = nn.MSELoss()
test_movie_idx = 14284


def run(game:cine2Nerdle,agent:Cine2NerdleAgent,start_movie_indexes=movie_idxs,tconst_subset=[],batch_size=1):
    #make start_movie_index batches
    batches = [start_movie_indexes[k:k+batch_size] for k in range(0,len(start_movie_indexes),batch_size)]
    #for batch in batches:
        #if len(batch) != batch_size:
            #print(f"{len(batch)}")
    #def subset_idxs_onehot(self,subset_idxs):
    #batches = [game.subset_idxs_onehot(batch) for batch in batches]
    #print(batches[0].shape)
    #print(batches[-1].shape)
    epochs = 1
    for epoch in range(1,epochs+1):
        print(f"Epoch {epoch}")
        for batch in batches:
            bs = len(batch)
            print(batch)
            ########START OF STATE INITIALIZATION
            #batch is a list of start movie indexes
            #from here we need to initialize the batch of match variables
            subset_indices = game.generate_subset_idxs(tconst_subset=tconst_subset,batch_size=bs)
            #this takes formulabest300 which is a list of tconsts, and maps the
            #tconsts to their respective indices in tconst_universe
            #print(f"subset_indices has shape = {subset_indices.shape}, type={subset_indices.dtype}")
            subset_onehot = game.subset_idxs_onehot(subset_indices)
            
            #this takes the tensor of movie indices and converts them to vectors
            #of length len(tconst_universe) which is 0 everywhere except the 
            #indices from subset_indices
            #print(f"subset_onehot has shape = {subset_onehot.shape}, type={subset_onehot.dtype}")

            moviesPlayed = test_game.init_moviesPlayed(batch)
            
            #print(f"moviesPlayed has shape = {moviesPlayed.shape}, type={moviesPlayed.dtype}")
            #print(moviesPlayed.sum(dim=1))

            actorsPlayed = test_game.init_actorsPlayed(batch_size=bs)
            
            #print(f"actorsPlayed has shape = {actorsPlayed.shape}, type={actorsPlayed.dtype}")
            #print(actorsPlayed.sum(dim=1))
            #print(actorsPlayed.sum(dim=1).shape)

            previousMoviePlayed = test_game.init_previousMoviePlayed(batch=batch)
            #print(f"previousMoviePlayed has shape = {previousMoviePlayed.shape}, type={previousMoviePlayed.dtype}")
            #print(f"previousMoviePlayed has sums = {previousMoviePlayed.sum(dim=1)}")
            #game_over = [False for _ in range(bs)]
            game_over = torch.zeros(bs).bool()
            ########END OF STATE INITIALIZATION
            #at this point we are ready to run the game itself for the batch
            iters = 0
            while False in game_over:
                #first we run the agent on the game state
                output = agent(subset=subset_onehot,moviesPlayed=moviesPlayed
                            ,actorsPlayed=actorsPlayed,previousMoviePlayed=previousMoviePlayed)
                
                subset_cast_onehot = game.onehot_casts[subset_indices].to(dtype=torch.float)
                subset_movie_onehot = F.one_hot(subset_indices,num_classes=len(tconst_universe)).to(torch.float)
                #print(subset_cast_onehot.shape)
                #print(subset_movie_onehot.shape)
                valid_move_mask,cdps = subset_valid_mask(subset_cast_onehots=subset_cast_onehot,
                                                    subset_movie_onehots=subset_movie_onehot,
                                                    previousMoviePlayed=previousMoviePlayed,
                                                    moviesPlayed=moviesPlayed,
                                                    actorsPlayed=actorsPlayed)
                valid_move_mask = torch.cat((valid_move_mask,torch.ones((bs,1))),dim=1).to(torch.bool)
                #print(valid_move_mask.sum(dim=1))
                #print(masked_tensor(output,valid_move_mask).argmax(dim=1).to(torch.int))
                #break
                print([int(subset_indices[0][i0]) for i0 in cdps.argmax(dim=1).tolist()])
                move = masked_tensor(output,valid_move_mask).argmin(dim=1).to(torch.int)
                #move = torch.argmax(output,dim=1)
                #print(move)
                move_ints = []
                for i in range(bs):
                    move_ints.append(int(move[i]))
                print(move_ints)
                #move_cast_ohs = game.movie_cast_onehot(torch.tensor(move_ints,dtype=actorsPlayed.dtype))
                
                move_movie_ohs = torch.zeros((bs,len(game.tconst_universe)),dtype=moviesPlayed.dtype)
                for i in range(move_movie_ohs.size(0)):
                    move_movie_ohs[i][move_ints[i]] = 1
                #diff = torch.sub(torch.amax(output,dim=1),torch.amin(output,dim=1))
                #print(diff)
                
                #one_hot_move = torch.zeros_like(output)
                #one_hot_move.scatter_(1,move.unsqueeze(1),1)
                #print(f"move has shape = {move.shape}, type={move.dtype}")
                #print(move)
                move_cast_ohs = move_cast_onehots(move=torch.tensor(move_ints,dtype=torch.long),subset_indices=subset_indices,onehot_casts=game.onehot_casts)
                #print(f"move_cast_ohs has shape = {move_cast_ohs.shape}, type={move_cast_ohs.dtype}")
                #print(move_cast_ohs.sum(dim=1))

                #move_movie_ohs = move_movie_onehots(move=move,subset_indices=subset_indices,onehot_casts=game.onehot_casts)
                #print(f"move_movie_ohs = {move_movie_ohs.shape}, type={move_movie_ohs.dtype}")
                
                for idx in range(bs):
                    if move_ints[idx] != subset_indices.size(1):
                        print(f"updated state {idx}")
                        previousMoviePlayed[idx] = move_cast_ohs[idx]
                        moviesPlayed[idx] = torch.add(moviesPlayed[idx],move_movie_ohs[idx])
                        actorsPlayed[idx] = torch.add(actorsPlayed[idx],move_cast_ohs[idx])
                        print(f"moviesPlayed sum {moviesPlayed[idx].sum()}")
                        print(f"moviesPlayed max {moviesPlayed[idx].max()}")
                iters += 1
                if iters > 300:
                    break
                else:
                    continue
                #break
                A = batch_dot_product(move_cast_ohs, previousMoviePlayed).bool()
                #this is a bool tensor of shape [batch_size]
                #True if the movie played by the agent connects to the previous movie
                #to obtain it we take the dot product of the cast onehot vectors
                #for the previous movie and the agents' move. non-zero dotprod->connection made
                #print(A.shape)

                B = torch.logical_not(batch_dot_product(move_movie_ohs,moviesPlayed).bool())
                #this is a bool tensor of shape [batch_size]
                #True if the movie that the agent picked has not already been played
                #obtained by dotproduct between chosen movie onehot and moviesPlayed
                #dotprod > 0 -> movie already played -> added logical_not in front
                #print(B.shape)

                C = ((move_cast_ohs+actorsPlayed) > 3).any(dim=1)
                #this is a bool tensor of shape [batch_size]
                #True if the movie that the agent picked would not push any values
                #in actors played past 3
                #print(C.shape)

                D = good_or_bad_concession(move=move, subset_indices=subset_indices
                                           , subset_onehot=subset_onehot, moviesPlayed=moviesPlayed
                                           ,actorsPlayed=actorsPlayed,previousMoviePlayed=previousMoviePlayed)
                #True->
                #print(D.shape)
                #print(D)

                E = torch.cat((A.view((1,bs)),B.view((1,bs)),C.view((1,bs)),D.view((1,bs))),dim=0)
                #print(E.shape)
                G = torch.logical_and(A.view((1,bs)),B.view((1,bs)))
                G = torch.logical_and(G,C.view((1,bs))).to(dtype=torch.float)
                #G = torch.cat((F,D.view((1,bs))),dim=0).to(dtype=torch.float)
                rewards2 = torch.matmul(torch.tensor([3e-3],dtype=torch.float),G)
                #print(G.shape)
                #print(rewards2)
                
                #print(E)
                reward_setting = torch.tensor([1e-3,1e-3,1e-3,-3e-3],dtype=torch.float)
                E = E.to(dtype=torch.float)
                rewards = torch.matmul(reward_setting,E)
                #print(rewards)
                rewards = torch.add(rewards,rewards2)
                #print(rewards)
                log_probs = torch.log_softmax(output,dim=1)
                #print(log_probs)
                selected_log_probs = (log_probs * one_hot_move).sum(dim=1)
                #print(selected_log_probs)
                loss = -(selected_log_probs * rewards).sum()
                print(loss.shape)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                for idx in range(bs):
                    if rewards[idx] >= 3:
                        print(f"updated state {idx}")
                        previousMoviePlayed[idx] = move_cast_ohs[idx]
                        moviesPlayed[idx] = torch.add(moviesPlayed[idx],move_movie_ohs[idx])
                        actorsPlayed[idx] = torch.add(actorsPlayed[idx],move_movie_ohs[idx])
                    
                # rewards.requires_grad_()
                # loss = loss_fn(rewards,torch.zeros_like(rewards,dtype=rewards.dtype))
                # loss.backward()
                # optimizer.step()
                #print(rewards)
                #print(rewards.sum())
                #rewards.backward()
                


            

            break
    return

run(game=test_game,agent=agent
    ,start_movie_indexes=repSubset
    ,tconst_subset=formulabest300
    ,batch_size=10)


In [ ]:
#https://github.com/google-deepmind/open_spiel/tree/master

In [5]:
def move_dot_products(s1_casts_oh:torch.Tensor,s2_casts_oh:torch.Tensor
                      ,s1_movies_oh:torch.Tensor,s2_movies_oh:torch.Tensor
                      ,pMP:torch.Tensor,aP:torch.Tensor,mP:torch.Tensor)->torch.Tensor:
    #print("###move_dot_products###")
    #s1/s2 _casts_oh are shape [subset_size, n_actors]
    #s1/s2 _movies_oh are shape [subset_size, n_movies]

    #pMP and aP have shape [batch_size, n_actors]
    #mP has shape [batch_size, n_movies]

    #a) compute cast dot products with pMP
    #s1_casts_oh has shape [subset_size, n_actors]
    #pMP has shape [batch_size, n_actors]
    #we want shape [batch_size,subset_size]
    #where each element is the dot product of each of the subset_size vectors of length n_actors in s1_casts_oh
    #with each of the batch_size vectors of length n_actors in pMP
    s1_casts_dotprod = torch.matmul(pMP,s1_casts_oh.T)
    s2_casts_dotprod = torch.matmul(pMP,s2_casts_oh.T)
    #print_shape_type(s1_casts_dotprod,"s1_casts_dotprod")
    #print_shape_type(s2_casts_dotprod,"s2_casts_dotprod")
    #b) compute dot products between each member of the batch's moviesPlayed (mP)
    #   and each of the subset_size movie onehot vectors (s1_movies_oh)
    #   values in the result will be 1 if s1_movies_oh[i] is in mP[i], 0 otherwise
    #   since we want 1s where s1_movies_oh[i] is not in mP[i], we negate
    #s1_movies_oh has shape [subset_size, n_movies]
    #mP has shape [batch_size, n_movies]
    #our output will be a tensor of 1s and 0s of shape [batch_size,subset_size]
    s1_already_played = torch.matmul(mP,s1_movies_oh.T)
    s1_already_played = torch.ones_like(s1_already_played) - s1_already_played
    s2_already_played = torch.matmul(mP,s2_movies_oh.T)
    s2_already_played = torch.ones_like(s2_already_played) - s2_already_played
    #print_shape_type(s1_already_played,"s1_already_played")
    #print_shape_type(s2_already_played,"s2_already_played")
    
    #c) add each of the cast onehot vectors in s1_casts_oh to each of the
    #   actorsPlayed vectors in aP.
    #   s1_casts_oh has shape [subset_size, n_actors]
    #   aP has shape [batch_size, n_actors]
    #   the values in the result of shape [batch_size,subset_size] should be
    #   1 if adding s1_casts_oh[i] to aP[j] results in a vector with no values > 3
    #   0 otherwise
    expanded_aP_s1 = aP.unsqueeze(1).expand(-1, s1_casts_oh.size(0), -1)
    expanded_aP_s2 = aP.unsqueeze(1).expand(-1, s2_casts_oh.size(0), -1)
    expanded_s1_casts_oh = s1_casts_oh.unsqueeze(0).expand(aP.size(0), -1, -1)
    expanded_s2_casts_oh = s2_casts_oh.unsqueeze(0).expand(aP.size(0), -1, -1)
    s1_satisfies_xs = ((expanded_aP_s1 + expanded_s1_casts_oh) <= 3).all(dim=2).to(torch.long)
    s2_satisfies_xs = ((expanded_aP_s2 + expanded_s2_casts_oh) <= 3).all(dim=2).to(torch.long)
    #print_shape_type(s1_satisfies_xs,"s1_satisfies_xs")
    #print_shape_type(s2_satisfies_xs,"s2_satisfies_xs")

    m1 = s1_already_played * s1_satisfies_xs
    m2 = s2_already_played * s2_satisfies_xs

    s1_casts_dotprod = s1_casts_dotprod * m1
    s2_casts_dotprod = s2_casts_dotprod * m2
    #print_shape_type(s1_casts_dotprod,"s1_casts_dotprod")
    #print_shape_type(s2_casts_dotprod,"s2_casts_dotprod")
    #print("###move_dot_products###")
    return s1_casts_dotprod,s2_casts_dotprod

def print_shape_type(t:torch.Tensor,tn:str)->None:
    print(f"{tn}: shape={t.shape}, type={t.dtype}")
    return
def subset_idxs_onehot(subset_indices):
    result = F.one_hot(subset_indices,num_classes=len(tconst_universe))
    return result
def argmin_nz(tensor:torch.Tensor)->torch.Tensor:
    mx = tensor.max()+1
    masked = tensor.clone()
    masked[masked == 0] = mx
    return torch.argmin(masked,dim=1)

In [4]:
def move_dot_products_optimized(s1_casts_oh: torch.Tensor, s2_casts_oh: torch.Tensor,
                                s1_movies_oh: torch.Tensor, s2_movies_oh: torch.Tensor,
                                pMP: torch.Tensor, aP: torch.Tensor, mP: torch.Tensor) -> torch.Tensor:
    # a) compute cast dot products with pMP
    #   pMP.shape = [batch_size,n_actors]
    #   sn_casts_oh.shape = [subset_size,n_actors]
    #   outputs.shape = [batch_size,subset_shape]
    s1_casts_dotprod = torch.matmul(pMP, s1_casts_oh.T)
    s2_casts_dotprod = torch.matmul(pMP, s2_casts_oh.T)

    # b) compute dot products and negate
    #   mP.shape = [batch_size,n_movies]
    #   sn_movies_oh.shape = [subset_size,n_movies]
    #   outputs.shape = [batch_size,subset_size]
    s1_already_played = 1 - torch.matmul(mP, s1_movies_oh.T)
    s2_already_played = 1 - torch.matmul(mP, s2_movies_oh.T)

    # c) check if satisfies condition
    #   aP.shape = [batch_size,n_actors]
    #   sn_casts_oh.shape = [subset_size,n_actors]
    #   outputs.shape = [batch_size,subset_size]
    s1_satisfies_xs = ((aP.unsqueeze(1) + s1_casts_oh) <= 3).all(dim=2).to(torch.long)
    s2_satisfies_xs = ((aP.unsqueeze(1) + s2_casts_oh) <= 3).all(dim=2).to(torch.long)

    # Apply masks
    s1_casts_dotprod = torch.where(s1_already_played * s1_satisfies_xs != 0, s1_casts_dotprod, torch.zeros_like(s1_casts_dotprod))
    s2_casts_dotprod = torch.where(s2_already_played * s2_satisfies_xs != 0, s2_casts_dotprod, torch.zeros_like(s2_casts_dotprod))

    return s1_casts_dotprod, s2_casts_dotprod

In [7]:
print(torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

True


In [12]:
onehot_casts = onehot_casts.float().to(device=device)
def run_matchup(s1:list,s2:list,batch_size:int=20,verbose:bool=False,device='cpu')->dict:
    #convert lists of tconsts to tensors of indices
    s1_indices=torch.tensor([tconst_idx[x] for x in s1],dtype=torch.long,device=device)
    s2_indices=torch.tensor([tconst_idx[x] for x in s2],dtype=torch.long,device=device)
    if verbose:
        print_shape_type(s1_indices,"s1_indices")
        print_shape_type(s2_indices,"s2_indices")
        print()
    #convert tensors of indices to tensors of onehots
    s1_casts_oh = onehot_casts[s1_indices].float().to(device=device)
    s2_casts_oh = onehot_casts[s2_indices].float().to(device=device)
    if verbose:
        print_shape_type(s1_casts_oh,"s1_casts_oh")
        print_shape_type(s2_casts_oh,"s2_casts_oh")
        print()
    s1_movies_oh = subset_idxs_onehot(s1_indices).float().to(device=device)
    s2_movies_oh = subset_idxs_onehot(s2_indices).float().to(device=device)
    if verbose:
        print_shape_type(s1_movies_oh,"s1_movies_oh")
        print_shape_type(s2_movies_oh,"s2_movies_oh")
        print()
    #get all start movie indices
    start_movie_indices = deepcopy(repSubset)
    if verbose:
        print(f"start_movie_indices type={type(start_movie_indices)}, len={len(start_movie_indices)}")
        print()
    #initialize looping helper variables
    cur_start_movies = [0 for _ in range(batch_size)]
    game_over = [True for _ in range(batch_size)]
    cur_n_moves = [0 for _ in range(batch_size)]
    if verbose:
            print(f"cur_start_movies type={type(cur_start_movies)}, len={len(cur_start_movies)}")
            print(f"game_over type={type(game_over)}, len={len(game_over)}")
            print()
    #initialize game variables
    pMP = torch.zeros((batch_size,len(nconst_universe)),dtype=torch.float,device=device)
    aP = torch.zeros((batch_size,len(nconst_universe)),dtype=torch.float,device=device)
    mP = torch.zeros((batch_size,len(tconst_universe)),dtype=torch.float,device=device)
    if verbose:
        print_shape_type(pMP,"pMP")
        print_shape_type(aP,"aP")
        print_shape_type(mP,"mP")
        print()
    result = {"s1":0,"s2":0}
    winners = {}
    ct = 0
    #start loop
    while len(start_movie_indices) > 0:
        #slot in new games for the ones that are completed
        if True in game_over:
            for i in range(len(game_over)):#check each slot
                if game_over[i] and cur_n_moves[i] == 0 and len(start_movie_indices) > 0:#initial fill
                    cur_start_movies[i] = start_movie_indices.pop()#assign a start movie to i
                    pMP[i] = onehot_casts[cur_start_movies[i]].clone()#init pMP to cast of start movie
                    aP[i] = torch.zeros_like(aP[i])#reset aP to zeros
                    mP[i] = F.one_hot(torch.tensor(cur_start_movies[i],dtype=torch.long),len(tconst_universe)).float()
                    #print(mP[i].sum())#verify one 1 per
                    #print(torch.nonzero(mP[i]))#verify location of 1
                    game_over[i] = False
                elif game_over[i]:
                    #record score of finished game
                    #if it was s1's turn and it couldnt make a move, then cur_n_moves[i] is even
                    #so, s2 wins if cur_n_moves[i] is even and game_over[i] is True
                    who_won = "s2" if cur_n_moves[i] % 2 == 0 else "s1"
                    if cur_start_movies[i] not in winners.keys():
                        cur_start_movies[i] = who_won
                    #recording game outcomes like this prevents us from doublecounting
                    #the outcome of the finished games that remain in the batch
                    #while the last games are finishing
                    #replace finished game with one that is on stack
                    if len(start_movie_indices) > 0:
                        cur_start_movies[i] = start_movie_indices.pop()
                        pMP[i] = deepcopy(onehot_casts[cur_start_movies[i]])
                        aP[i] = torch.zeros_like(aP[i])
                        mP[i] = F.one_hot(torch.tensor(cur_start_movies[i],dtype=torch.long),len(tconst_universe)).float()
                        game_over[i] = False
                        cur_n_moves[i] = 0
                    
        if verbose:
            print_shape_type(pMP,"pMP")
            print_shape_type(aP,"aP")
            print_shape_type(mP,"mP")
            print()
        
        #compute moves
        s1_out,s2_out = move_dot_products_optimized(s1_casts_oh,s2_casts_oh,s1_movies_oh,s2_movies_oh,pMP,aP,mP)
        # print_shape_type(s1_out,"s1_out")
        # print_shape_type(s2_out,"s2_out")
        s1_has_vms = (s1_out.sum(dim=1) == 0).tolist()
        s2_has_vms = (s2_out.sum(dim=1) == 0).tolist()
        # print(s1_has_vms)
        # print(s2_has_vms)
        s1_moves = [int(s1_indices[x]) for x in argmin_nz(s1_out).tolist()]
        s2_moves = [int(s2_indices[x]) for x in argmin_nz(s2_out).tolist()]
        # print(cur_start_movies)
        # print(s1_moves)
        # print(s2_moves)
        
        #apply moves
        for i in range(len(s1_moves)):
            #print(f"before: mP.sum={mP[i].sum()}, aP.sum={aP[i].sum()}, pMP.sum={pMP[i].sum()}")
            if cur_n_moves[i] % 2 == 0 and not s1_has_vms[i]:
                #apply move s1_moves[i] to game[i]
                cur_n_moves[i]+=1
                #s1_moves[i] is an index in tconst_universe
                #update mP
                mP[i][s1_moves[i]] = 1.0
                #update aP
                aP[i] = aP[i] + onehot_casts[s1_moves[i]]
                #update pMP
                pMP[i] = onehot_casts[s1_moves[i]].clone()
            elif cur_n_moves[i] % 2 == 1 and not s2_has_vms[i]:
                #apply move s2_moves[i] to game[i]
                cur_n_moves[i]+=1
                #s2_moves[i] is an index in tconst_universe
                #update mP
                mP[i][s2_moves[i]] = 1.0
                #update aP
                aP[i] = aP[i] + onehot_casts[s2_moves[i]]
                #update pMP
                pMP[i] = onehot_casts[s2_moves[i]].clone()
            elif cur_n_moves[i] % 2 == 0:
                #game over at s1's turn
                game_over[i] = True
                print(f"game{ct} finished in {cur_n_moves[i]} moves")
                ct+=1
            elif cur_n_moves[i] % 2 == 1:
                #game over at s2's turn
                game_over[i] = True
                print(f"game{ct} finished in {cur_n_moves[i]} moves")
                ct+=1
            #print(f"after: mP.sum={mP[i].sum()}, aP.sum={aP[i].sum()}, pMP.sum={pMP[i].sum()}")
        #break
    for k,v in winners.items():
        result[v] += 1
    return winners

test = run_matchup(s1=formulabest300,s2=formulabest500,batch_size=20,verbose=False,device=device)
print(test)

game0 finished in 91 moves
game1 finished in 93 moves
game2 finished in 93 moves
game3 finished in 97 moves
game4 finished in 109 moves
game5 finished in 109 moves
game6 finished in 116 moves
game7 finished in 117 moves
game8 finished in 142 moves
game9 finished in 146 moves
game10 finished in 149 moves
game11 finished in 154 moves
game12 finished in 155 moves
game13 finished in 159 moves
game14 finished in 160 moves
game15 finished in 160 moves
game16 finished in 161 moves
game17 finished in 162 moves
game18 finished in 162 moves
game19 finished in 162 moves
game20 finished in 93 moves
game21 finished in 91 moves
game22 finished in 124 moves
game23 finished in 142 moves
game24 finished in 155 moves
game25 finished in 154 moves
game26 finished in 129 moves
game27 finished in 154 moves
game28 finished in 112 moves
game29 finished in 168 moves
game30 finished in 91 moves
game31 finished in 150 moves
game32 finished in 141 moves
game33 finished in 154 moves
game34 finished in 146 moves
ga